In [ ]:
!pip install -q finance-datareader

In [ ]:
import FinanceDataReader as fdr

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def make_stock_df(market = None, name = None, symbol = None, start_date = '2021', end_date = None, high = 80, low = 20):
    """
    market = 'KOSPI', 'KOSDAQ' ...etc
    name = 상장된 주식 이름
    symbol = 주식 코드 또는 해외 주식 이름
    start_date = 해당 주식에 가져오고 싶은 시작일
    end_date = 해당 주식에 가져오고 싶은 종료일
    """

    if market != None and name != None:
        stock_list = fdr.StockListing(market)
        symbol = stock_list.loc[stock_list['Name'] == name]['Code']

    df = fdr.DataReader(symbol, start = start_date, end = end_date)   # 코드, 시작날짜, 종료 날짜

    # 단순 이동평균선
    df['SMA_5'] = df['Close'].rolling(window = 5).mean()
    df['SMA_20'] = df['Close'].rolling(window = 20).mean()
    df['SMA_60'] = df['Close'].rolling(window = 60).mean()
    df['SMA_120'] = df['Close'].rolling(window = 120).mean()

    # 볼린저 밴드
    df['SMA_20_std'] = df['Close'].rolling(window = 20).std()
    df['Upper_Band'] = df['SMA_20'] + (2 * df['SMA_20_std'])
    df['Lower_Band'] = df['SMA_20'] - (2 * df['SMA_20_std'])

    # pct_b 컬럼 생성
    df['pct_b'] = (df['Close'] - df['Lower_Band']) / (df['Upper_Band'] - df['Lower_Band'])

    # Band_Width 컬럼 생성
    df['Band_Width'] = (df['Upper_Band'] - df['Lower_Band']) / (df['SMA_20'])

    # 자금 흐름 지표
    df['TP'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['PMF'] = 0
    df['NMF'] = 0

    for i in range(len(df) - 1):
        if df['TP'].values[i] <= df['TP'].values[i + 1]:
            df['PMF'].values[i + 1] = df['TP'].values[i + 1] * df['Volume'].values[i + 1]
            df['NMF'].values[i + 1] = 0

        elif df['TP'].values[i] > df['TP'].values[i + 1]:
            df['PMF'].values[i + 1] = 0
            df['NMF'].values[i + 1] = df['TP'].values[i + 1] * df['Volume'].values[i + 1]
    df['MFI'] = 100 - 100 / (1 + df['PMF'].rolling(window = 14).sum() / df['NMF'].rolling(window = 14).sum())

    def signal(data):
        buy = []
        sell = []
        for i in range(len(data['MFI'])):
            if data['MFI'][i] > high: # 매도 타이밍
                buy.append(np.NaN)
                sell.append(data['Close'][i])
            elif data['MFI'][i] < low:  # 매수 타이밍
                buy.append(data['Close'][i])
                sell.append(np.NaN)
            else:
                buy.append(np.NaN)
                sell.append(np.NaN)

        return buy, sell

    df['Buy'], df['Sell'] = signal(df)

    # MACD

    df['MACD_Short'] = df['Close'].ewm(span = 12).mean()
    df['MACD_Long'] = df['Close'].ewm(span = 26).mean()
    df['MACD'] = df['MACD_Short'] - df['MACD_Long']
    df['MACD_Signal'] = df['MACD'].ewm(span = 9).mean()
    df['MACD_Sign'] = df.apply(lambda x : ("매수" if x['MACD'] > x['MACD'] else "매도"), axis = 1)
    df['MACD_Oscillator'] = df['MACD'] - df['MACD_Signal']

    # 수익률
    df['Close_pct'] = 100 * df['Close'].pct_change(periods = 1)
    return df

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

def stock_visualization(df):
    fig = make_subplots(
        rows=3,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1,
        row_heights=[0.5, 0.25, 0.25],
    )

    # Close
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['Close'],
        mode='lines',
        name='Close',
        line=dict(color='rgb(255,255,255)', width=2)
    ), row=1, col=1)

    # MA 5
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['SMA_5'],
        mode='lines',
        name='MA_5',
        line=dict(color='rgb(255,255,0)', width=1)
    ), row=1, col=1)

    # MA 20
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['SMA_20'],
        mode='lines',
        name='MA_20',
        line=dict(color='rgb(255,0,127)', width=1)
    ), row=1, col=1)

    # MA 60
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['SMA_60'],
        mode='lines',
        name='MA_60',
        line=dict(color='rgb(0,255,0)', width=1)
    ), row=1, col=1)

    # MA 120
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['SMA_120'],
        mode='lines',
        name='MA_120',
        line=dict(color='rgb(0,0,255)', width=1)
    ), row=1, col=1)

    # 상단 밴드 라인
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['Upper_Band'],
        mode='lines',
        name='Upper_Band',
        line=dict(color='darkgray', width = 1)
    ), row=1, col=1)

    # 하단 밴드 라인
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['Lower_Band'],
        mode='lines',
        name='Lower_Band',
        line=dict(color='darkgray', width = 1)
    ), row=1, col=1)

    # 밴드 사이 영역 채우기
    fig.add_trace(go.Scatter(
        x=df.index.tolist() + df.index[::-1].tolist(),  # x축 데이터를 두 배로 확장 (역순 포함)
        y=df['Upper_Band'].tolist() + df['Lower_Band'][::-1].tolist(),  # y축 데이터를 두 배로 확장 (역순 포함)
        fill='toself',  # 자기 자신까지 채우기
        fillcolor='rgba(128, 128, 128, 0.2)',  # 밴드 색상 (투명도 포함)
        line=dict(color='gray'),
        name='band_fill'
    ), row=1, col=1)

##############################################################################################

    # MACD 라인
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['MACD'],
        mode='lines',
        name='MACD'
    ), row=2, col=1)

    # MACD Signal 라인
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['MACD_Signal'],
        mode='lines',
        name='MACD_Signal'
    ), row=2, col=1)

    # MACD 오실레이터 막대
    fig.add_trace(go.Bar(
        x=df[df['MACD_Oscillator'] >= 0].index,
        y=df[df['MACD_Oscillator'] >= 0]['MACD_Oscillator'],
        name='MACD_Oscillator_Positive',
        marker_color='red'
    ), row=2, col=1)

    fig.add_trace(go.Bar(
        x=df[df['MACD_Oscillator'] < 0].index,
        y=df[df['MACD_Oscillator'] < 0]['MACD_Oscillator'],
        name='MACD_Oscillator_Negative',
        marker_color='blue'
    ), row=2, col=1)

##############################################################################################

    # MFI
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['MFI'],
        mode='lines',
        name='MFI',
        line=dict(color='orange')
    ), row=3, col=1)

    fig.add_trace(go.Scatter(
        x=[df.index.min(), df.index.max()],
        y=[80, 80],
        mode='lines',
        name='Overbought',
        line=dict(color='blue', dash='dash')
    ), row=3, col=1)

    fig.add_trace(go.Scatter(
        x=[df.index.min(), df.index.max()],
        y=[20, 20],
        mode='lines',
        name='Oversold',
        line=dict(color='red', dash='dash')
    ), row=3, col=1)

    # 레이아웃 업데이트
    fig.update_layout(
        title='Bollinger Bands & MACD & MFI',
        xaxis3_title='Date',  # 두 번째 행의 x축 제목
        yaxis_title='Close',  # 첫 번째 행의 y축 제목
        yaxis2_title='MACD',  # 두 번째 행의 y축 제목
        yaxis3_title='MFI',  # 두 번째 행의 y축 제목

        legend_title='Legend',
        template='plotly_dark',
        xaxis_rangeslider_visible=True,  # RangeSlider 추가
        xaxis2_rangeslider_visible=True,  # RangeSlider 추가
        xaxis3_rangeslider_visible=True,  # RangeSlider 추가

        height=1200 # 전체 화면 높이 조정
    )

    # x축 범위 선택기 추가
    fig.update_xaxes(
        rangeslider=dict(
            visible=True,
            thickness=0.1,
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ]),
            bgcolor='rgba(150, 150, 150, 0.5)',  # 배경색
            font=dict(color='black')  # 텍스트 색상
        )
    )
    # MFI y축 범위 설정
    fig.update_yaxes(range=[0, 100], row=3, col=1)

    pio.show(fig, config={'responsive': True, 'displayModeBar': True, 'modeBarButtonsToAdd': ['toggleSpikelines'], 'scrollZoom': True, 'displaylogo': False})


    fig.write_html('stock_visualization.html', auto_open = True)

In [ ]:
data = make_stock_df(symbol = 'AAPL')
# data = make_stock_df(market = 'KOSPI', name = '삼성전자')

In [ ]:
stock_visualization(data)